In [1]:
import warnings
# warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from ML_Pipelines import *

from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, log_loss, recall_score, plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn import set_config
set_config(display='diagram')
from sklearn.utils import estimator_html_repr


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

sns.set_style("darkgrid")
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('payoneer_data.csv').sample(20000)

## Pandas & SKlearn

In [3]:
y = df['label']
X = df.drop(['label'], axis=1).copy()

In [4]:
X_train_pre, X_test_pre, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

In [6]:
X_train_pre

,feature_2,feature_3,feature_4,feature_15,feature_18,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_16,feature_17,feature_19,feature_20,feature_21
185115,21665,2493,35,117,3,d9750ee7192,ecad2386ff,07d7df22v8n,9c7b6730eei7,0,1005,0,-1,7801e8d91e,a0fa4be313p,1,98572c794,320,f028772b7,a99f214ay39
192711,21665,2493,35,117,3,85f751fd192,3c4b944dff,0f2161f8v8n,820bcf17eei7,0,1005,0,-1,2347f47a1e,0170201513p,1,c4e18dd64,320,50e219e07,e75fac87y39
171739,22713,2627,1059,110,3,85f751fd192,1dc72b4dff,0f2161f8v8n,8803c3d6eei7,2,1005,1,100013,2347f47a1e,c355d98913p,1,c4e18dd64,300,50e219e07,d462d8bby39
29850,21189,2424,161,71,1,85f751fd192,92f5800bff,0f2161f8v8n,981edffceei7,3,1005,0,100193,ae6375221e,dc34853013p,1,c4e18dd64,320,50e219e07,a99f214ay39
3683,19771,2227,935,48,0,856e6d3f192,ecad2386ff,07d7df22v8n,e12e75cdeei7,0,1005,1,100074,7801e8d91e,56b9001413p,1,58a89a434,320,f028772b7,a99f214ay39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146895,15703,1722,35,79,0,1fbe01fe192,ecad2386ff,07d7df22v8n,fbec56bceei7,0,1005,0,-1,7801e8d91e,077de38313p,1,f38457674,320,28905ebd7,a99f214ay39
54181,15703,1722,35,79,0,85f751fd192,03528b27ff,8ded1f7av8n,b78e6543eei7,0,1005,0,100083,2347f47a1e,665810f313p,1,c4e18dd64,320,50e219e07,a99f214ay39
200869,21684,2496,167,23,3,15941585192,ecad2386ff,07d7df22v8n,072c9f1eeei7,0,1002,0,-1,7801e8d91e,e8b8ed7613p,0,a562b6554,320,50e219e07,5c23e5dey39
9225,13777,1447,47,13,2,0bf7cf2a192,ecad2386ff,07d7df22v8n,d787e91beei7,0,1005,0,100079,7801e8d91e,26eb3a6013p,1,e6221ce34,320,3e8141307,a99f214ay39


## Find optimal model using baseline models

In [15]:
data_pipeline = pipeline()
X_train = data_pipeline.fit_transform(X_train_pre)
X_test = data_pipeline.transform(X_test_pre)

shape pre transform (10000, 9)
shape after transform (10000, 9)
shape pre transform (10000, 9)
shape after transform (10000, 9)


In [ ]:
sum(X_train[:, -1]==1)

In [ ]:
sum(X_train[:, -1]==-1)

In [ ]:
classifiers = [
    LogisticRegression(class_weight='balanced', max_iter=1000),
#     SVC(kernel="rbf", C=0.025, probability=True, class_weight='balanced'),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
#     HistGradientBoostingRegressor(),
    XGBClassifier()
    ]

In [ ]:
eval_clf(classifiers, X_train, X_test, y_train, y_test)

In [ ]:
params_clf = {'scale_pos_weight' : (y_train==False).sum()/(y_train==True).sum(),
              'eval_metric' : "logloss", # 'error'
              'random_state':1}

clf = XGBClassifier(**params_clf)

In [ ]:
param_grid = { 
            'min_child_weight': [0.5],
            'learning_rate':[0.1, 0.35],
            'n_estimators':[300],
            }

options = {#'early_stopping_rounds' : 10,
            'verbose' : 100,
           'refit':'f1',
           'cv' : 3,
           'scoring' : ['roc_auc', 'f1', 'recall', 'precision']
          }

gscv, bst, results_df = grid_search2(clf, param_grid, X_train, X_test, y_train, y_test, **options)

In [ ]:
param_grid = { 
            'min_child_weight': [0.5],
            'learning_rate':[0.1, 0.35],
            'n_estimators':[300],
            }

options = {'early_stopping_rounds' : 10,
            'verbose' : 100,
           'refit':'f1',
           'scoring' : ['roc_auc', 'f1', 'recall', 'precision']
          }

gscv, bst, results_df = grid_search(clf, param_grid, X_train, X_test, y_train, y_test, **options)

In [ ]:
df_cv_results

In [ ]:
bst_clf = bst.fit(X_train, y_train)
bst_clf.n_estimators = bst_clf.best_iteration

bias_var_plot(bst_clf)

In [ ]:
pipeline_clf = Pipeline(steps=[('data_pipeline', data_pipeline), 
                               ('classifier', gscv.best_estimator_
                               )])

In [ ]:
pipeline_clf

In [ ]:
clf = pipeline_clf.fit(X_train_pre, y_train)

In [ ]:
cv_score = cross_val_score(clf, X_train_pre, y_train, cv=3, error_score=0)
print('After grid search train set cross validation mean score {:.4f}'.format(np.mean(cv_score)))

In [ ]:
plot_confusion_matrix(bst_clf, X_test, y_test, normalize='all', cmap='summer')  
plt.rcParams["figure.figsize"] = (30, 10)
plt.rcParams["font.size"] =22
plt.show()